In [76]:
### IMPORTS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
# from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras import backend
from keras import optimizers
import keras
import folium
from folium.plugins import AntPath, MousePosition
from sklearn.utils import shuffle, resample
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Dropout, LSTM, Input, RepeatVector, GRU, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
from datetime import datetime
import random
import math
from tqdm import tqdm
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from sklearn.metrics import silhouette_score
from scipy.stats import gaussian_kde
import seaborn as sns
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon


In [ ]:
df = pd.read_csv(r"D:\Python\master thesis\collision data\final_dataset_collision.csv")
df = df.drop(columns=['Unnamed: 0'])
df['# Timestamp'] = pd.to_datetime(df['# Timestamp'])
df = df.sort_values(by=['MMSI', '# Timestamp'])
original_data = df
minutes_before_collision = 3
plt.plot(df[df['MMSI'] == 219021240.0][-(30+minutes_before_collision):-minutes_before_collision]['Longitude'],df[df['MMSI'] == 219021240.0][-(30+minutes_before_collision):-minutes_before_collision]['Latitude'])
plt.plot(df[df['MMSI']== 232018267.0][:147][-(30+minutes_before_collision):-minutes_before_collision]['Longitude'],df[df['MMSI']== 232018267.0][:147][-(30+minutes_before_collision):-minutes_before_collision]['Latitude'])
for i in range(30):
    if i%2 ==0 :
        color = 'red'
    else:
        color = 'blue' 
    longs = [df[df['MMSI'] == 219021240.0][-(30+minutes_before_collision):-minutes_before_collision][i:i+1]['Longitude'],df[df['MMSI']== 232018267.0][:147][-(30+minutes_before_collision):-minutes_before_collision][i:i+1]['Longitude']]
    lats = [df[df['MMSI'] == 219021240.0][-(30+minutes_before_collision):-minutes_before_collision][i:i+1]['Latitude'],df[df['MMSI']== 232018267.0][:147][-(30+minutes_before_collision):-minutes_before_collision][i:i+1]['Latitude']]
    print(haversine_distance(df[df['MMSI'] == 219021240.0][-(30+minutes_before_collision):-minutes_before_collision][i:i+1]['Longitude'],df[df['MMSI'] == 219021240.0][-(30+minutes_before_collision):-minutes_before_collision][i:i+1]['Latitude'],df[df['MMSI']== 232018267.0][:147][-(30+minutes_before_collision):-minutes_before_collision][i:i+1]['Longitude'],df[df['MMSI']== 232018267.0][:147][-(30+minutes_before_collision):-minutes_before_collision][i:i+1]['Latitude']))
    plt.plot(longs,lats,alpha = 0.3,linewidth=0.4,color = color)
# plt.plot(df[df['MMSI'] == 219021240]['Longitude'],df[df['MMSI'] == 219021240]['Latitude'],zorder=1)
# plt.plot(df[df['MMSI'] == 232018267][50:147]['Longitude'],df[df['MMSI'] == 232018267][50:147]['Latitude'],zorder=2)
# plt.scatter(df[df['MMSI'] == 219021240][-2:-1]['Longitude'],df[df['MMSI'] == 219021240][-2:-1]['Latitude'],200,color='red',marker=(10, 1, 0),zorder=3,)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

In [ ]:
### DATASET LOAD

df = pd.read_csv(r"D:\Python\master thesis\collision data\final_dataset_collision.csv")
df = df.drop(columns=['Unnamed: 0'])
df['# Timestamp'] = pd.to_datetime(df['# Timestamp'])
df = df.sort_values(by=['MMSI', '# Timestamp'])


def diff_to_coord(y,X):
    a = np.zeros_like(y)
    for j in tqdm(range(a.shape[0])):
        for i in range(a.shape[1]):
            if i == 0:
                a[j,i,0] = X[j,-1,0] - y[j,i,0]
                a[j,i,1] = X[j,-1,1] - y[j,i,1]
            else:
                a[j,i,0] = a[j,i-1,0] - y[j,i,0]
                a[j,i,1] = a[j,i-1,1] - y[j,i,1]
    return a
def sequences_from_collision(minutes_before_collision):
    seq = []
    seq.append(df[df['MMSI'] == 219021240.0][-(30+minutes_before_collision):-minutes_before_collision])
    seq.append(df[df['MMSI']== 232018267.0][:147][-(30+minutes_before_collision):-minutes_before_collision])
    seq = np.dstack(seq)
    seq = np.rollaxis(seq,-1)
    return seq
def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r
def euclidean_distance(lon1,lat1,lon2,lat2):
    return math.sqrt((lon1-lon2)**2+(lat1-lat2)**2)

seq = sequences_from_collision(3)

bad_sequences = []
for index,i in tqdm(enumerate(seq)):
    # print(index)
    if 0 in i[:,4] or (i[:,6] > 2).sum() > 0 or (i[:,6] < 1).sum() > 0 or 0 in (np.append(np.atleast_2d(i[:,7:9].sum(axis=1)).T,i[:,7:8],axis=1)).sum(axis=1) or np.isnan((i[:,2:].astype(np.float32))).sum() > 0 or (i[:,11] <= 0).sum() > 0 or (i[:,11] > 800).sum() > 0:
        bad_sequences.append(index)
new_seq = np.delete(seq,bad_sequences,axis=0)
new = new_seq.reshape((new_seq.shape[0]*new_seq.shape[1],new_seq.shape[2]))
df = pd.DataFrame(new,columns=['MMSI','# Timestamp','Latitude', 'Longitude' ,'SOG','Heading','DateDiff','Lat_speed','Long_speed','Lat_lag','Long_lag','haversine_distance'])

df['d_Lat'] = df['Lat_lag'] - df['Latitude']
df['d_Long'] = df['Long_lag'] - df['Longitude']

### NORMALIZING VALUES existing scaler
import joblib

small_df = df[['MMSI','# Timestamp']]
df = df.drop(columns=['MMSI','# Timestamp'])
features=[i for i in df.columns]

scaler = joblib.load('sc.joblib')

small_df[features] = scaler.transform(df[features])
df = small_df
df_numpy = df.to_numpy()
seq = df_numpy.reshape((new_seq.shape[0],new_seq.shape[1],df.shape[1]))

### DIVIDING INTO FEATURE (X) AND TARGET VARIABLES (y)
seq_input_length = 30
X_test = seq[:,0:seq_input_length,:]
X_test_primary = X_test
X_test = X_test[:,:,2:]
X_test=X_test.astype('float32')

seq_output_length = 20
model_names = []
predictions = []
for model_name in os.listdir('D:\Python\master thesis\master-thesis\models'):
    if '.h5' in model_name:
        print(model_name)
        model = load_model(f'D:\Python\master thesis\master-thesis\models\{model_name}')
        y_pred_test = model.predict(X_test)
        # print(y_pred_test.shape)
        y_pred_test= y_pred_test.reshape(y_pred_test.shape[0],20,2)
        y_pred_test = np.concatenate((np.zeros((y_pred_test.shape[0], y_pred_test.shape[1], X_test.shape[2] - y_pred_test.shape[2])),y_pred_test),axis=2)
        y_pred_test = scaler.inverse_transform(y_pred_test.reshape(y_pred_test.shape[0]*y_pred_test.shape[1],y_pred_test.shape[2]))[:,-2:]
        y_pred_test = y_pred_test.reshape((int(y_pred_test.shape[0]/seq_output_length),seq_output_length,2))
        print(y_pred_test.shape)
        model_names.append(model_name)
        predictions.append(y_pred_test)


X_test = scaler.inverse_transform(X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2]))
X_test = X_test.reshape((int(X_test.shape[0]/seq_input_length),seq_input_length,12))
# y_test = diff_to_coord(y_test,X_test)
predictions_new = []
for y_pred in predictions:
    y_pred_test = diff_to_coord(y_pred,X_test)
    predictions_new.append(y_pred_test)
predictions = predictions_new

predictions_new = []
for y_pred_test in tqdm(predictions):
    last_timestamps = X_test_primary[:, -1, 1]    
    y_pred_timestamps = []
    for seq_index, last_timestamp in enumerate(last_timestamps):
        timestamps = [last_timestamp + np.timedelta64(i + 1, 'm') for i in range(20)]
        # print(timestamps)
        y_pred_timestamps.append(timestamps)
    y_pred_timestamps = np.array(y_pred_timestamps)[..., np.newaxis]
    # print(y_pred_timestamps.shape)
    y_pred_test_with_timestamps = np.concatenate((y_pred_timestamps,y_pred_test), axis=-1)
    adjusted_identifier = X_test_primary[:,:20,0]
    adjusted_identifier = adjusted_identifier[..., np.newaxis]
    y_pred_test_with_identifier = np.concatenate((adjusted_identifier,y_pred_test_with_timestamps ), axis=-1)
    predictions_new.append(y_pred_test_with_identifier)    
predictions = predictions_new
new_predictions= np.stack(predictions,axis=-1)

total_interactions = 0
interactions = []
for i in tqdm(range(X_test_primary.shape[0])):
    for j in range(X_test_primary.shape[0]):
        if j > i and X_test_primary[i,0,0] != X_test_primary[j,0,0] and not(X_test_primary[i,-1,1] + np.timedelta64(19,'m') < X_test_primary[j,-1,1] or X_test_primary[j,-1,1] + np.timedelta64(19,'m') < X_test_primary[i,-1,1]):
            total_interactions+=1
            interactions.append([i,j,1])
                

# **Predictor**

In [96]:
def vizualize_ellipse(prediction_region,with_points=True,legend=True,label=True,vizualize=True):
    """
    Vizualizes confide ellipse around a bi-variate cluster

    Args:
        prediction_region : np.array of size (2,x) where x > 1, prediction_region[0] = Latitude, prediction_region[1] = Longitude
        with_points : additionally scatter plot with points and mean
    """

    t = np.linspace(0,2.0 * np.pi,1000)
    xy = np.stack((np.cos(t),np.sin(t)),axis=-1)
    covariance = np.cov(prediction_region[1,:].astype('float'),prediction_region[0,:].astype('float'))
    eigenvalues, eigenvectors = np.linalg.eig(covariance)
    val = np.sqrt(eigenvalues)
    center = np.mean([prediction_region[1,:].astype('float'),prediction_region[0,:].astype('float')],axis=1)[:,None]
    if vizualize:
        if with_points:
            plt.scatter(prediction_region[1,:],prediction_region[0,:],label='Predicted points',color='#1f77bd')
            plt.scatter(center[0],center[1],label='Mean prediction',color='#ff7f0e')
        if label:
            plt.plot(*(2.44765*eigenvectors @ (val * xy).T + center),label='Predicted trajectory regions',color='#2ca02c')
        else:
            plt.plot(*(2.44765*eigenvectors @ (val * xy).T + center),color='#2ca02c')
        # plt.plot(*(2.44765*eigenvectors @ (val * xy).T + center),label='95% confidence ellipse',color='#2ca02c')
        if legend:
            plt.legend(loc='upper right')
        plt.ticklabel_format(useOffset=False)
        plt.gca().locator_params(nbins=5)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
    else:
        pass
    return covariance,eigenvectors,val,eigenvalues,center

def point_intersection(scatter1,scatter2,vizualize=False):
    # plt.scatter(scatter1[3,:],scatter1[2,:])
    # # plt.scatter(scatter2[3,:],scatter2[2,:])
    ## Point intersection
    covariance1 = np.cov(scatter1[3,:].astype('float'),scatter1[2,:].astype('float'))
    covariance2 = np.cov(scatter2[3,:].astype('float'),scatter2[2,:].astype('float'))
    
    eigenvalues1, eigenvectors1 = np.linalg.eig(covariance1)
    eigenvalues2, eigenvectors2 = np.linalg.eig(covariance2)
    val1 = np.sqrt(eigenvalues1)
    val2 = np.sqrt(eigenvalues2)
    center1 = np.mean([scatter1[3,:].astype('float'),scatter1[2,:].astype('float')],axis=1)[:,None]
    center2 = np.mean([scatter2[3,:].astype('float'),scatter2[2,:].astype('float')],axis=1)[:,None]
    intersection_points = 0
    for point_idx in range(scatter1.shape[1]):
        point1 = scatter1[2:4,point_idx]
        point1 = np.flip(point1)
        point2 = scatter2[2:4,point_idx]
        point2 = np.flip(point2)
        # plt.scatter(point[1],point[0])
        if (point1 - center2.flatten()) @ (np.linalg.inv((5.991)*covariance2)) @ (point1 - center2.flatten()) < 1:
            intersection_points += 1 #skaiciuojam ar point in ar out ellipse 
            plt.scatter(point1[0],point1[1],color='red')
        if (point2 - center1.flatten()) @ (np.linalg.inv((5.991)*covariance1)) @ (point2 - center1.flatten()) < 1:
            intersection_points += 1 #skaiciuojam ar point in ar out ellipse
            plt.scatter(point2[0],point2[1],color='red')
    if intersection_points > 0 and vizualize == True:
        vizualize_ellipse(scatter1[2:4,:],True,False,False)
        vizualize_ellipse(scatter2[2:4,:],True,False,False)
    return intersection_points

def ellipsoidal_area_intersection(scatter1,scatter2):
    covariance1 = np.cov(scatter1[3,:].astype('float'),scatter1[2,:].astype('float'))
    covariance2 = np.cov(scatter2[3,:].astype('float'),scatter2[2,:].astype('float'))
    eigenvalues1, eigenvectors1 = np.linalg.eig(covariance1)
    eigenvalues2, eigenvectors2 = np.linalg.eig(covariance2)
    val1 = np.sqrt(eigenvalues1)
    val2 = np.sqrt(eigenvalues2)
    center1 = np.mean([scatter1[3,:].astype('float'),scatter1[2,:].astype('float')],axis=1)[:,None]
    center2 = np.mean([scatter2[3,:].astype('float'),scatter2[2,:].astype('float')],axis=1)[:,None]
    ellipse1 = 2.44765*eigenvectors1 @ (val1 * xy).T + center1
    ellipse2 = 2.44765*eigenvectors2 @ (val2 * xy).T + center2
    polygon1 = Polygon(ellipse1.T)
    polygon2 = Polygon(ellipse2.T)
    intersection = polygon1.intersection(polygon2)
    if not intersection.is_empty:
        
        intersection_area = intersection.area
        # print(f"The area of intersection is: {intersection_area}")
        # print(f"Intersection percentage {round((intersection_area*100)/(polygon1.area + polygon2.area - intersection_area),2)}%")
        iou = (intersection_area*100)/(polygon1.area + polygon2.area - intersection_area)
        intersection_x,intersection_y = intersection.exterior.xy
        plt.fill(intersection_x,intersection_y,alpha=0.4,color='y')
        
        # inset_position = [0.65, 0.65, 0.25, 0.25]  # [left, bottom, width, height] in figure coordinates
        # inset_ax = plt.axes(inset_position)  # Create inset axes
        # inset_ax.ticklabel_format(useOffset=False)
        # inset_ax.fill(intersection_x,intersection_y,alpha=0.4,color='y')
        # inset_ax.set_xlim(12.2139, 12.2142)
        # inset_ax.set_ylim(54.5513, 54.5514)
        # # inset_ax.set_xticklabels([])
        # inset_ax.set_title("Zoomed in intersection", fontsize=10)
        # inset_ax.tick_params(labelsize=8)  # Smaller ticks for inset

        return intersection.area,iou
    else:
        return 0,0

def z_score_bounding_box(vessel_cluster, z_score):
    vessel1_long_m,vessel1_lat_m = np.mean([vessel_cluster[3,:],vessel_cluster[2,:]],axis=1)
    vessel1_long_std,vessel1_lat_std = np.std(np.array(vessel_cluster[3,:])),np.std(np.array(vessel_cluster[2,:]))
    
    vessel1_long_lim_min = vessel1_long_m  - vessel1_long_std*z_score
    vessel1_long_lim_max = vessel1_long_m  + vessel1_long_std*z_score
    vessel1_lat_lim_min = vessel1_lat_m  - vessel1_lat_std*z_score
    vessel1_lat_lim_max = vessel1_lat_m  + vessel1_lat_std*z_score
    coord1 = [vessel1_long_lim_min,vessel1_lat_lim_min]
    coord2 = [vessel1_long_lim_min,vessel1_lat_lim_max]
    coord3 = [vessel1_long_lim_max,vessel1_lat_lim_max]
    coord4 = [vessel1_long_lim_max,vessel1_lat_lim_min]
    square_coord = np.vstack((coord1,coord2,coord3,coord4,coord1)).T
    return square_coord

def square_intersection_area(square1,square2):
    sq1_b_l =np.min(square1,axis=1)
    sq1_t_r= np.max(square1,axis=1)
    sq2_b_l =np.min(square2,axis=1)
    sq2_t_r= np.max(square2,axis=1)
    x_l = max(sq1_b_l[0],sq2_b_l[0])
    y_b = max(sq1_b_l[1],sq2_b_l[1])
    x_r = min(sq1_t_r[0],sq2_t_r[0])
    y_t = min(sq1_t_r[1],sq2_t_r[1])
    coord1 = [x_l,y_b]
    coord2 = [x_l,y_t]
    coord3 = [x_r,y_t]
    coord4 = [x_r,y_b]
    if x_l < x_r and y_b < y_t:
        width = x_r - x_l
        height = y_t - y_b
        area = width * height
        sq1_area = (sq1_t_r[0] - sq1_b_l[0]) * (sq1_t_r[1] - sq1_b_l[1])
        sq2_area = (sq2_t_r[0] - sq2_b_l[0]) * (sq2_t_r[1] - sq2_b_l[1])
        iou = area/(sq1_area+sq2_area-area)
        intersect_coords = np.vstack((coord1,coord2,coord3,coord4,coord1)).T
        return area,iou,intersect_coords
    else:
        # print('Squares do not intersect')
        return 0,0,[]
    
def gmm(scatter1,scatter2):
    np.random.seed(1)
    cluster_1 = scatter1[2:4].T  # Cluster 1 (centered at [-2, -2])
    cluster_2 = scatter2[2:4].T    # Cluster 2 (centered at [2, 2])

    # # Combine the clusters into one array
    data = np.vstack([cluster_1, cluster_2])
    dataframe = pd.DataFrame(data, columns=['Latitude', 'Longitude'])
    gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=42)
    gmm.fit(data)
    labels = gmm.predict(data)  # Cluster assignments
    labels = np.array([0] * 10 + [1] * 10)
    dataframe['Cluster'] = labels
    g = sns.JointGrid(data=dataframe, x='Longitude', y='Latitude')
    g.ax_joint.ticklabel_format(useOffset=False)

    # Add scatter points for each cluster with different colors
    colors = ['red', 'blue']
    for cluster_id, color in enumerate(colors):
        cluster_data = dataframe[dataframe['Cluster'] == cluster_id]
        g.ax_joint.scatter(
            cluster_data['Longitude'],
            cluster_data['Latitude'],
            label=f'Cluster {cluster_id + 1}',
            color=color,
            s=20,
            alpha=1
    )

    kde1_long = gaussian_kde(np.array(cluster_1[:,1],dtype='float'))
    kde2_long = gaussian_kde(np.array(cluster_2[:,1],dtype='float'))
    kde1_lat = gaussian_kde(np.array(cluster_1[:,0],dtype='float'))
    kde2_lat = gaussian_kde(np.array(cluster_2[:,0],dtype='float'))

    x_long = np.linspace(min(min(cluster_1[:,1]), min(cluster_2[:,1]))-0.001, max(max(cluster_1[:,1]), max(cluster_2[:,1]))+0.001, 1000)
    x_lat = np.linspace(min(min(cluster_1[:,0]), min(cluster_2[:,0]))-0.001, max(max(cluster_1[:,0]), max(cluster_2[:,0]))+0.001, 1000)



    kde1_values_long = kde1_long(x_long)
    kde2_values_long = kde2_long(x_long)

    kde1_values_lat = kde1_lat(x_lat)
    kde2_values_lat = kde2_lat(x_lat)

    g.ax_marg_x.plot(x_long, kde1_values_long, label='Cluster Data 1', lw=2,color=colors[0])
    g.ax_marg_x.plot(x_long, kde2_values_long, label='Cluster Data 2', lw=2,color=colors[1])
    g.ax_marg_x.fill_between(x_long, kde1_values_long, alpha=0.2, label='Intersection',color='red')
    g.ax_marg_x.fill_between(x_long, kde2_values_long, alpha=0.2, label='Intersection',color='blue')
    g.ax_marg_x.fill_between(x_long, np.minimum(kde1_values_long, kde2_values_long), alpha=0.5, label='Intersection',color='yellow')

    g.ax_marg_y.plot(kde1_values_lat,x_lat, label='Cluster Data 1', lw=2,color=colors[0])
    g.ax_marg_y.plot(kde2_values_lat,x_lat, label='Cluster Data 2', lw=2,color=colors[1])
    g.ax_marg_y.fill_betweenx(x_lat,kde1_values_lat, alpha=0.2, label='Intersection',color='red')
    g.ax_marg_y.fill_betweenx(x_lat,kde2_values_lat, alpha=0.2, label='Intersection',color='blue')
    g.ax_marg_y.fill_betweenx(x_lat,np.minimum(kde1_values_lat, kde2_values_lat), alpha=0.5, label='Intersection',color='yellow')
    
    intersection_area_long = np.trapz(np.minimum(kde1_values_long, kde2_values_long), x_long)
    intersection_area_lat = np.trapz(np.minimum(kde1_values_lat, kde2_values_lat), x_long)
    print(f"Longitude intersection area {intersection_area_long}")
    print(f"Latitude intersection area {intersection_area_lat}")
    print(f"Intersection area multiplied {intersection_area_long*intersection_area_lat}")
    print('------')
    plt.show()
    return intersection_area_long*intersection_area_lat

### ELLIPSOIDAL POINT INTERSECTION

In [ ]:

plt.ticklabel_format(useOffset=False)
t = np.linspace(0,2.0 * np.pi,1000)
xy = np.stack((np.cos(t),np.sin(t)),axis=-1)
for interaction in tqdm(interactions):
    plt.plot(X_test[interaction[0],:,1],X_test[interaction[0],:,0]) #first ship predictor
    plt.plot(X_test[interaction[1],:,1],X_test[interaction[1],:,0]) #second ship predictor
    
    intersection = np.intersect1d(y_pred_test_with_identifier[interaction[0],:,1],y_pred_test_with_identifier[interaction[1],:,1])
    palette = sns.color_palette("viridis_r", len(intersection))
    for i,timepoint in enumerate(intersection):
        plt.ticklabel_format(useOffset=False)
        plt.gca().locator_params(nbins=5)
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        plt.scatter(scatter1[3,:],scatter1[2,:],color=palette[i],alpha=0.5)
        plt.scatter(scatter2[3,:],scatter2[2,:],color=palette[i],alpha=0.5)
        covariance1,eigenvectors1,val1,eigenvalues1,center1 = vizualize_ellipse(scatter1[2:4,:],False,False,False,False)
        covariance2,eigenvectors2,val2,eigenvalues2,center2 = vizualize_ellipse(scatter2[2:4,:],False,False,False,False)
        plt.plot(*(2.44765*eigenvectors1 @ (val1 * xy).T + center1),color=palette[i])
        plt.plot(*(2.44765*eigenvectors2 @ (val2 * xy).T + center2),color=palette[i])
        intersection_points = point_intersection(scatter1,scatter2,False)
        print(intersection_points)
        # plt.show()
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
plt.plot(original_data[original_data['MMSI']== 232018267.0][:147][-3:]['Longitude'],original_data[original_data['MMSI']== 232018267.0][:147][-3:]['Latitude'],color='red',alpha=0.5)
plt.plot(original_data[original_data['MMSI'] == 219021240.0][-3:]['Longitude'],original_data[original_data['MMSI'] == 219021240.0][-3:]['Latitude'],color='orange',alpha=0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

### ELLIPSOIDAL AREA INTERSECTION

In [ ]:
plt.ticklabel_format(useOffset=False)
t = np.linspace(0,2.0 * np.pi,1000)
xy = np.stack((np.cos(t),np.sin(t)),axis=-1)
for interaction in tqdm(interactions):
    # plt.plot(X_test[interaction[0],:,1],X_test[interaction[0],:,0]) #first ship predictor
    # plt.plot(X_test[interaction[1],:,1],X_test[interaction[1],:,0]) #second ship predictor
    
    intersection = np.intersect1d(y_pred_test_with_identifier[interaction[0],:,1],y_pred_test_with_identifier[interaction[1],:,1])
    palette = sns.color_palette("viridis_r", len(intersection))
    for i,timepoint in enumerate(intersection):
        if i == 3:
            pass
        else:
            continue
        plt.ticklabel_format(useOffset=False)
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        plt.scatter(scatter1[3,:],scatter1[2,:],color=palette[i],alpha=0.5)
        plt.scatter(scatter2[3,:],scatter2[2,:],color=palette[i],alpha=0.5)
        covariance1,eigenvectors1,val1,eigenvalues1,center1 = vizualize_ellipse(scatter1[2:4,:],False,False,False,False)
        covariance2,eigenvectors2,val2,eigenvalues2,center2 = vizualize_ellipse(scatter2[2:4,:],False,False,False,False)
        plt.plot(*(2.44765*eigenvectors1 @ (val1 * xy).T + center1),color=palette[i])
        plt.plot(*(2.44765*eigenvectors2 @ (val2 * xy).T + center2),color=palette[i])
        # intersection_points = point_intersection(scatter1,scatter2,False)
        area,iou = ellipsoidal_area_intersection(scatter1,scatter2)
        print(area,iou)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.xlim([14.2428,14.2431])
        plt.ylim([55.22235,55.2226])
        plt.show()
        # if area > 0:
        #     print(interaction)
        #     print(scatter1[:,0])
        #     print(scatter2[:,0])
        
        
    

### Z-SCORE INTERSECTION AREA

In [ ]:
## Z-score intersection area

for interaction in tqdm(interactions):

    plt.plot(X_test[interaction[0],:,1],X_test[interaction[0],:,0]) #first ship predictor
    plt.plot(X_test[interaction[1],:,1],X_test[interaction[1],:,0]) #second ship predictor
    
    intersection = np.intersect1d(y_pred_test_with_identifier[interaction[0],:,1],y_pred_test_with_identifier[interaction[1],:,1])
    palette = sns.color_palette("viridis_r", len(intersection))
    for i,timepoint in enumerate(intersection):
        plt.ticklabel_format(useOffset=False)
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        plt.scatter(scatter1[3,:],scatter1[2,:],color=palette[i],alpha=0.5)
        plt.scatter(scatter2[3,:],scatter2[2,:],color=palette[i],alpha=0.5)
        square_coord1 = z_score_bounding_box(scatter1,3)
        square_coord2 = z_score_bounding_box(scatter2,3)
        area, iou,intersect_coords= square_intersection_area(square_coord1,square_coord2)
        # plt.scatter(y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='red')
        # plt.scatter(y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='orange')
        # print(area,iou)
        plt.plot(square_coord1[0],square_coord1[1],color=palette[i])
        plt.plot(square_coord2[0],square_coord2[1],color=palette[i])
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        if area > 0:
            plt.fill(intersect_coords[0],intersect_coords[1],color = 'y',alpha= 0.3)
        print(iou)
plt.plot(original_data[original_data['MMSI']== 232018267.0][:147][-10:]['Longitude'],original_data[original_data['MMSI']== 232018267.0][:147][-10:]['Latitude'],color='red',alpha=0.5)
plt.plot(original_data[original_data['MMSI'] == 219021240.0][-10:]['Longitude'],original_data[original_data['MMSI'] == 219021240.0][-10:]['Latitude'],color='orange',alpha=0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

### GMM

In [ ]:
for interaction in tqdm(interactions):

    intersection = np.intersect1d(y_pred_test_with_identifier[interaction[0],:,1],y_pred_test_with_identifier[interaction[1],:,1])
    # if interaction[2] == 1:
    highest_per_interaction = 0
    for i,timepoint in enumerate(intersection):
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        
        # plt.scatter(scatter1[3,:],scatter1[2,:],color=palette[i],alpha=0.5)
        # plt.scatter(scatter2[3,:],scatter2[2,:],color=palette[i],alpha=0.5)
        score = gmm(scatter1,scatter2)
        # plt.scatter(y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='red')
        # plt.scatter(y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='orange')
        if score > highest_per_interaction:
            highest_per_interaction = score
    print(highest_per_interaction)
        

### SILHOUETTE VALUE

In [ ]:
## Silhouette
silhouette_interactions = []
for interaction in tqdm(interactions):

    intersection = np.intersect1d(y_pred_test_with_identifier[interaction[0],:,1],y_pred_test_with_identifier[interaction[1],:,1])
    # if interaction[0] == 2351 and interaction[1] == 8929:
    #     pass
    # else:
    #     continue
    palette = sns.color_palette("viridis_r", len(intersection))
    lowest_per_interaction = 1
    for i,timepoint in enumerate(intersection):
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        cluster1_points = scatter1[2:4,:].T
        cluster2_points = scatter2[2:4,:].T
        combined_data = np.vstack([cluster1_points, cluster2_points])
        labels = np.array([0] * cluster1_points.shape[0] + [1] * cluster2_points.shape[0])
        silhouette_avg = silhouette_score(combined_data, labels)
        # print(f"Silhoutte power score: {(1-silhouette_avg)**(1./20)}")
        # print(f"Silhouette Value: {silhouette_avg}")
        if silhouette_avg < lowest_per_interaction:
            lowest_per_interaction = silhouette_avg
        # print(1-silhouette_avg)
        print(n_root(1 - silhouette_avg,10))
        
        # print(f"Silhoutte logistic score: {(1-(1./(1+math.e**(-12*(silhouette_avg-0.5)))))}")

        # plt.scatter(scatter1[3,:],scatter1[2,:],color=palette[i],alpha=0.5)
        # plt.scatter(scatter2[3,:],scatter2[2,:],color=palette[i],alpha=0.5)
        # plt.scatter(y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='red')
        # plt.scatter(y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='orange')
    silhouette_interactions.append([1-lowest_per_interaction,interaction[2]])
    

    # print(highest_per_interaction)
# gmm_interactions = np.array(gmm_interactions)    

In [72]:
def n_root(x,n):
    return x**(1./n)    
def n_logistic(x,n):
    return (1-(1./(1+math.e**(n*(x-0.5)))))